# Export/Import Fiddler Models

Export complete model definitions from one Fiddler instance and import them to another, with support for:
* Complete model schema with column definitions (types, ranges, categories)
* Model spec (inputs, outputs, targets, metadata, decisions)
* Task configuration and parameters
* Custom features for LLM models (enrichments)
* Baselines (rolling baselines - static baselines require datasets)
* Related assets (segments, custom metrics)
* Cross-instance transfers (different Fiddler instances)
* Model versioning (create new versions of existing models)
* No DataFrame required for import (uses Model constructor pattern)

## Current Support Status

**Fully Supported:**
* Model schema, spec, and task configuration
* Rolling baselines (auto-created)
* Segments (full import with FQL definitions)
* Custom metrics (full import with definitions)

**Partial Support:**
* Static baselines (exported but require manual dataset publishing before creation)

**Not Yet Supported:**
* Alerts (exported for reference but require manual creation due to metric ID mapping)
* Model artifacts (must be manually re-uploaded after import)

## Prerequisites

* Source Fiddler instance URL and API token
* Target Fiddler instance URL and API token (can be same as source)
* Source model must exist (target model will be created)
* `fiddler_utils` package installed (run cell below)

In [1]:
# Install dependencies
%pip install -q fiddler-client

# Install fiddler_utils from parent directory
# If not already installed, run from repo root: pip install -e .
import sys
sys.path.insert(0, '..')

Note: you may need to restart the kernel to use updated packages.


In [2]:
import fiddler as fdl
from fiddler_utils import (
    ConnectionManager,
    ModelManager,
)

print(f'Fiddler client version: {fdl.__version__}')
print('fiddler_utils: Successfully imported')

Fiddler client version: 3.10.0
fiddler_utils: Successfully imported


## Configuration

In [3]:
# Source Fiddler Instance
SOURCE_URL = "https://demo.fiddler.ai"  # e.g., 'https://source.fiddler.ai'
SOURCE_TOKEN = "bviEN-TNK5LhJ-ObZNMboMCnm99LxG7eVSDwjJzX_es"
SOURCE_PROJECT_NAME = "bank_churn"
SOURCE_MODEL_NAME = "churn_classifier"
SOURCE_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Target Fiddler Instance (can be same as source)
TARGET_URL = "https://preprod.cloud.fiddler.ai"  # e.g., 'https://target.fiddler.ai'
TARGET_TOKEN = "hqvUV7r8-WUkMkjvKHbvI_sVpxRd9DJLKX6PCloRwVk"
TARGET_PROJECT_NAME = "migration_util_demo"
# Target model name will be taken from export (or can be modified before import)

# Export Options
INCLUDE_BASELINES = True  # Include baseline definitions
INCLUDE_RELATED_ASSETS = True  # Include segments, metrics, alerts

# Import Options
IMPORT_MODE = 'create_new'  # 'create_new', 'create_version', or 'update_existing'
VERSION_LABEL = 'v2'  # Required if IMPORT_MODE='create_version'
CREATE_BASELINES = True  # Recreate baselines on target
IMPORT_RELATED_ASSETS = True  # Import segments, metrics, alerts

## Setup Connections and Initialize Manager

In [4]:
# Setup connection manager for handling multiple instances
conn_mgr = ConnectionManager(log_level='WARNING')
conn_mgr.add('source', url=SOURCE_URL, token=SOURCE_TOKEN)
conn_mgr.add('target', url=TARGET_URL, token=TARGET_TOKEN)

# Initialize ModelManager
model_mgr = ModelManager()

print('✓ Connection manager configured')
print('✓ ModelManager initialized')

✓ Connection manager configured
✓ ModelManager initialized


## Fetch Source Model

In [5]:
# Connect to source and get model
with conn_mgr.use('source'):
    source_project = fdl.Project.from_name(SOURCE_PROJECT_NAME)
    source_model_kwargs = {'project_id': source_project.id, 'name': SOURCE_MODEL_NAME}
    if SOURCE_MODEL_VERSION:
        source_model_kwargs['version'] = SOURCE_MODEL_VERSION
    
    source_model = fdl.Model.from_name(**source_model_kwargs)
    print(f'Source model: {source_model.name} (ID: {source_model.id})')
    if source_model.version:
        print(f'  Version: {source_model.version}')

Source model: churn_classifier (ID: 8dea99c0-6724-46df-a2de-5d542ed7f272)
  Version: v1


## Model Export and Import Workflow

The following cells demonstrate the complete workflow for exporting and importing models.

**Note:** Import examples are commented out to avoid creating duplicate models during testing.
Uncomment the import cells to actually perform imports.

### Display Model Information

View comprehensive model details including schema, columns, and related assets.

In [6]:
with conn_mgr.use('source'):
    model_mgr.display_model_info(
        model_id=source_model.id,
        show_columns=True,
        show_related_assets=True
    )


Model: churn_classifier
Version: v1
Task: binary_classification
Project ID: 1eaf9070-af0c-4d50-a3c4-1f7968bd0574
Model ID: 8dea99c0-6724-46df-a2de-5d542ed7f272

Event Columns:
  Timestamp Column: __occurred_at

Task Parameters:
  Target Class Order: ['no', 'yes']
  Binary Threshold: 0.5

Schema: 13 columns
  Inputs: 9
  Outputs: 1
  Targets: 1
  Metadata: 1
  Decisions: 1

------------------------------------------------------------
Columns:
------------------------------------------------------------
  creditscore                     INPUT       INTEGER       min=350  max=850
  geography                       INPUT       CATEGORY      categories=['California', 'Florida', 'Hawaii', ... (6 total)]
  age                             INPUT       INTEGER       min=18  max=90
  tenure                          INPUT       INTEGER       min=0  max=10
  balance                         INPUT       FLOAT         min=0  max=222267.63
  numofproducts                   INPUT       INTEGER       min

### Export Model

Export complete model definition including all related assets.

In [7]:
print("\n" + "=" * 60)
print("EXPORTING MODEL")
print("=" * 60)

with conn_mgr.use('source'):
    model_export = model_mgr.export_model(
        model_id=source_model.id,
        include_baselines=True,
        include_related_assets=True
    )

print(f"\n✓ Exported model: {model_export.name}")
if model_export.version:
    print(f"  Version: {model_export.version}")
print(f"  Task: {model_export.task}")
print(f"  Columns: {len(model_export.columns)}")
print(f"  Baselines: {len(model_export.baselines)}")
print(f"  Custom features: {len(model_export.custom_features)}")

# Show related assets counts
print("\n  Related assets:")
for asset_type, assets in model_export.related_assets.items():
    print(f"    {asset_type}: {len(assets)}")

# Artifact warning
if model_export.has_artifacts:
    print("\n  ⚠️  Model has uploaded artifacts (must be re-uploaded after import)")

# Show sample column info
print("\n  Sample columns (first 3):")
for col in model_export.columns[:3]:
    info = f"    {col.name} ({col.data_type})"
    if col.categories:
        info += f" - {len(col.categories)} categories"
    elif col.min_value is not None:
        info += f" - range: [{col.min_value}, {col.max_value}]"
    print(info)


EXPORTING MODEL

✓ Exported model: churn_classifier
  Version: v1
  Task: binary_classification
  Columns: 13
  Baselines: 7
  Custom features: 0

  Related assets:
    segments: 15
    custom_metrics: 9
    alerts: 6

  Sample columns (first 3):
    creditscore (INTEGER) - range: [350, 850]
    geography (CATEGORY) - 6 categories
    age (INTEGER) - range: [18, 90]


### Save Model Export to File

Save the exported model definition to JSON file for later use or archival.

In [8]:
import os

# Create exports directory if it doesn't exist
os.makedirs('exports', exist_ok=True)

# Save to file
export_filename = f"exports/{model_export.name}_export.json"
model_export.to_json(export_filename)

print(f"✓ Saved model export to: {export_filename}")
print(f"  File size: {os.path.getsize(export_filename) / 1024:.1f} KB")

# Can load it back later:
# loaded_export = ModelExportData.from_json(export_filename)

✓ Saved model export to: exports/churn_classifier_export.json
  File size: 22.0 KB


### Import Model to Target Project

Import the exported model to a different project (or instance) as a new model.

**Note:** This example is commented out to avoid creating duplicate models.
Uncomment to actually import.

In [10]:
print("\n" + "=" * 60)
print("IMPORTING MODEL (create_new mode)")
print("=" * 60)

with conn_mgr.use('target'):
    # Connect to target project and get Project reference
    target_project = fdl.Project.from_name(TARGET_PROJECT_NAME)

    # Import as new model
    # Will fail if model name already exists (safe default)
    imported_model = model_mgr.import_model(
        target_project_id=target_project.id,
        model_data=model_export,
        import_mode='create_new',
        create_baselines=True,
        import_related_assets=True
    )

    print(f"\n✓ Imported model: {imported_model.name}")
    print(f"  Model ID: {imported_model.id}")
    print(f"  Task: {imported_model.task}")
    print(f"  Columns: {len(imported_model.schema.columns)}")


IMPORTING MODEL (create_new mode)

Importing 7 baselines...
  ⊗ Skipped: default_static_baseline (STATIC)
  ⊗ Skipped: bank_churn (STATIC)
  ⊗ Skipped: bank_churn_baseline2 (STATIC)
  ⊗ Skipped: static_baseline (STATIC)
  ✓ Created: rolling_baseline_1week (ROLLING)
  ✓ Created: rolling_baseline_1day (ROLLING)
  ✓ Created: rolling_baseline_1month (ROLLING)
  Summary: 3 created, 4 skipped, 0 failed

Importing 15 segments (with validation)...
  ✓ Created: 15, Skipped: 0, Failed: 0

Importing 9 custom metrics (with validation)...
  ✓ Created: 9, Skipped: 0, Failed: 0

Importing 6 alerts...
  ⚠️  Alert import is not yet supported - metric IDs require manual mapping
  📋 Exported alert definitions are saved in the export data for reference

✓ Imported model: churn_classifier
  Model ID: 86a54295-09e2-43c4-b212-01c12c4563be
  Task: binary_classification
  Columns: 13


### Import as New Model Version

Import as a new version of an existing model using Fiddler's model versioning.

**Note:** This example is commented out to avoid creating duplicate versions.

In [11]:
# Uncomment to create a new version:

print("\n" + "=" * 60)
print("IMPORTING MODEL (create_version mode)")
print("=" * 60)

with conn_mgr.use('target'):
    # Import as new version of existing model
    versioned_model = model_mgr.import_model(
        target_project_id=target_project.id,
        model_data=model_export,
        import_mode='create_version',
        version_label='v2-automated',  # Required for create_version mode
        create_baselines=True,
        import_related_assets=True
    )

    print(f"\n✓ Created new version: {versioned_model.name}")
    print(f"  Version: {versioned_model.version}")
    print(f"  Model ID: {versioned_model.id}")


IMPORTING MODEL (create_version mode)

Importing 7 baselines...
  ⊗ Skipped: default_static_baseline (STATIC)
  ⊗ Skipped: bank_churn (STATIC)
  ⊗ Skipped: bank_churn_baseline2 (STATIC)
  ⊗ Skipped: static_baseline (STATIC)
  ✓ Created: rolling_baseline_1week (ROLLING)
  ✓ Created: rolling_baseline_1day (ROLLING)
  ✓ Created: rolling_baseline_1month (ROLLING)
  Summary: 3 created, 4 skipped, 0 failed

Importing 15 segments (with validation)...
  ✓ Created: 15, Skipped: 0, Failed: 0

Importing 9 custom metrics (with validation)...
  ✓ Created: 9, Skipped: 0, Failed: 0

Importing 6 alerts...
  ⚠️  Alert import is not yet supported - metric IDs require manual mapping
  📋 Exported alert definitions are saved in the export data for reference

✓ Created new version: churn_classifier
  Version: v2-automated
  Model ID: a5f75510-bdba-4022-9585-97c1d73fd535


### Model Management Summary

Key advantages of using ModelManager:

In [12]:
print("\n" + "=" * 60)
print("MODEL MANAGEMENT SUMMARY")
print("=" * 60)

print("\n✅ Advantages of ModelManager:\n")
print("  • Complete model export (schema, spec, task, baselines, related assets)")
print("  • Uses Model constructor (no DataFrame needed for import!)")
print("  • Deterministic and reproducible imports")
print("  • Automatic UUID resolution for related assets")
print("  • Multiple import modes (create_new, create_version, update_existing)")
print("  • Safe defaults (fails if model exists in create_new mode)")
print("  • JSON serialization for archival and version control")
print("  • Display and comparison utilities")

print("\n📊 Export Statistics:\n")
print(f"  Model: {model_export.name}")
print(f"  Columns: {len(model_export.columns)}")
print(f"  Inputs: {len(model_export.spec['inputs'])}")
print(f"  Outputs: {len(model_export.spec['outputs'])}")
print(f"  Targets: {len(model_export.spec['targets'])}")
print(f"  Baselines: {len(model_export.baselines)}")
print(f"  Custom features: {len(model_export.custom_features)}")

total_related = sum(len(assets) for assets in model_export.related_assets.values())
print(f"  Related assets: {total_related} total")
for asset_type, assets in model_export.related_assets.items():
    print(f"    - {asset_type}: {len(assets)}")

print("\n" + "=" * 60)


MODEL MANAGEMENT SUMMARY

✅ Advantages of ModelManager:

  • Complete model export (schema, spec, task, baselines, related assets)
  • Uses Model constructor (no DataFrame needed for import!)
  • Deterministic and reproducible imports
  • Automatic UUID resolution for related assets
  • Multiple import modes (create_new, create_version, update_existing)
  • Safe defaults (fails if model exists in create_new mode)
  • JSON serialization for archival and version control
  • Display and comparison utilities

📊 Export Statistics:

  Model: churn_classifier
  Columns: 13
  Inputs: 9
  Outputs: 1
  Targets: 1
  Baselines: 7
  Custom features: 0
  Related assets: 30 total
    - segments: 15
    - custom_metrics: 9
    - alerts: 6

